In [1]:
import sys
sys.path.append("../../")

In [2]:
from DataLoader.extra_features_dataset import FoodRatingsDataset
from torch.utils.data import DataLoader

from tqdm.notebook import tqdm
import math
import os
import pandas as pd
import numpy as np

## Load

In [3]:
from features_train_functions import fit

In [4]:
n_users = 226570
n_items = 231637

# batch_size = 4
batch_size = 4086

device = "cuda"

train_dataloader = DataLoader(
    FoodRatingsDataset(
        "../../Preprocessing/processed_dataframes/train.csv",
#         "../../Preprocessing/processed_dataframes/val.csv",
        "../../Preprocessing/processed_dataframes/sorted_recipes.csv",
        n_users,
        n_items,
        has_rating_column=True,
    ), 
    batch_size=batch_size,
    num_workers=8,
    shuffle=False,
)

val_dataloader = DataLoader(
    FoodRatingsDataset(
        "../../Preprocessing/processed_dataframes/val.csv",
#         "../../Preprocessing/processed_dataframes/test.csv",
        "../../Preprocessing/processed_dataframes/sorted_recipes.csv",
        n_users,
        n_items,
        has_rating_column=True,
    ), 
    batch_size=batch_size,
    num_workers=4,
    shuffle=False,
)

test_dataloader = DataLoader(
    FoodRatingsDataset(
        "../../Preprocessing/processed_dataframes/test.csv",
        "../../Preprocessing/processed_dataframes/sorted_recipes.csv",
        n_users,
        n_items,
        has_rating_column=True,
    ), 
    batch_size=batch_size,
    num_workers=12,
    shuffle=False,
)

### Run training

In [5]:
losses_fit = []
n_features = 10
n_txt = 120900

for run_number, (k_gmf, k_mlp, layers, f_layers, txt_layers, lr, epochs) in tqdm(enumerate([
#     (8, 8, [8], [10], 0.001, 20)
#     (4, 4, [4], [4], 0.001, 20)
#     (8, 8, [32, 16, 8], [16, 8], 0.001, 20)
    (8, 8, [32, 16, 8], [16, 8], [128, 64, 32, 16, 8], 0.001, 20)
])):
#     print(f"{k_gmf=} {k_mlp=} {layers=} {lr=} {epochs=}")
    losses_fit.append(
        fit(
            train_dataloader, 
            val_dataloader,
            n_users,
            n_items,
            n_features,
            n_txt,
            k_gmf,
            k_mlp,
            layers,
            f_layers,
            txt_layers,
            lr,
            epochs,
            weight_path="/home/nubol23/Documents/Project/NCF_dot",
            run_number=run_number,
            random_state=2,
            show_loss=True,
            device="cuda",
        )
    )

0it [00:00, ?it/s]

NeuMF(
  (P_gmf): Embedding(226570, 8)
  (Q_gmf): Embedding(231637, 8)
  (P_mlp): Embedding(226570, 8)
  (Q_mlp): Embedding(231637, 8)
  (fc_layers): ModuleList(
    (0): Linear(in_features=16, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=16, bias=True)
    (2): Linear(in_features=16, out_features=8, bias=True)
  )
  (feature_layers): ModuleList(
    (0): Linear(in_features=10, out_features=16, bias=True)
    (1): Linear(in_features=16, out_features=8, bias=True)
  )
  (txt_layers): ModuleList(
    (0): Linear(in_features=120900, out_features=128, bias=True)
    (1): Linear(in_features=128, out_features=64, bias=True)
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): Linear(in_features=32, out_features=16, bias=True)
    (4): Linear(in_features=16, out_features=8, bias=True)
  )
  (h): Linear(in_features=32, out_features=64, bias=True)
  (hn1): Linear(in_features=64, out_features=32, bias=True)
  (hn2): Linear(in_features=32, out_features=

  0%|          | 0/20 [00:00<?, ?it/s]

epoch: 1, train_loss: 2.9568098563458083, val_loss: 1.1029490747956099
epoch: 2, train_loss: 0.9437815954283735, val_loss: 0.9172706468278279
epoch: 3, train_loss: 0.8456319048242055, val_loss: 0.9214556320741775
epoch: 4, train_loss: 0.7938633875095306, val_loss: 0.9365415250723998
epoch: 5, train_loss: 0.7397600917930396, val_loss: 0.9526497408798764
epoch: 6, train_loss: 0.6748969381041525, val_loss: 0.9770772612359823


KeyboardInterrupt: 